In [1]:
import pandas as pd
import string
import numpy as np
import glob
from keras.preprocessing.sequence import pad_sequences
from nltk.stem import PorterStemmer

In [2]:
porter=PorterStemmer()
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']
import pickle
with open('tokenizer-200-nou.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
def clean_punct(a):        
    return a.lower().translate(str.maketrans('', '', string.punctuation))

def clean_stop(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
        if tk in stopwords:
            continue
        else:
            str1 += tk+" "
            
    return str1

def clean_stem(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
            str1 += porter.stem(tk)+" "
            
    return str1

In [4]:
from tensorflow import keras
model = keras.models.load_model('./Modele_LSTM/Model-200-l22-5')

count=0;

pozitive_total=0
negative_total=0
pozitive_corecte=0
negative_corecte=0
neutre_total=0
neutre_corecte=0
txtfiles = []
for file in glob.glob("Stiri_pozitive/*.txt"):
    txtfiles.append(file)
    

print("---Stiri Pozitive---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        paragrafe=list(map(clean_stem, paragrafe))
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
        
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
            if np.count_nonzero(X_train[i]) > 2:
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
        
        print(str(file)+"  "+str(sum_neg/nr_paragrafe)+"     "+str(sum_poz/nr_paragrafe))
        
        pozitive_total+=1
        if sum_neg < sum_poz:
            pozitive_corecte+=1

     
print("Total stiri pozitive : "+str(pozitive_total))
print("Corecte stiri pozitive : "+str(pozitive_corecte)) 
print("Acuratete Pozitive: "+str(pozitive_corecte/pozitive_total))
print("")    

txtfiles = []
for file in glob.glob("Stiri_negative/*.txt"):
    txtfiles.append(file)
print("---Stiri Negative---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        paragrafe=list(map(clean_stem, paragrafe))
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
        
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
            if np.count_nonzero(X_train[i]) > 2:
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
                
        print(str(file)+"  "+str(sum_neg/nr_paragrafe)+"     "+str(sum_poz/nr_paragrafe))
        
        negative_total+=1
        if sum_neg > sum_poz:
            negative_corecte+=1
    
print("Total stiri negative : "+str(negative_total))
print("Corecte stiri negative : "+str(negative_corecte)) 
print("Acuratete negative: "+str(negative_corecte/negative_total))

print("")
print("Total stiri :" + str(pozitive_total+negative_total+neutre_total))
print("Total corecte :" + str(pozitive_corecte+negative_corecte+neutre_corecte))
print("Acuratete :" + str((pozitive_corecte+negative_corecte+neutre_corecte)/(pozitive_total+negative_total+neutre_total)))
    
    

2022-03-02 16:55:10.048775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


2022-03-02 16:55:10.071812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-02 16:55:10.073046: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-02 16:55:10.075757: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-02 16:55:10.077378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

---Stiri Pozitive---
Stiri_pozitive/34.txt  0.43361177207529544     0.566388226767381
Stiri_pozitive/42.txt  0.566565046644462     0.43343494930810517
Stiri_pozitive/12.txt  0.40499603637543163     0.5950039691340766
Stiri_pozitive/41.txt  0.4675745585312446     0.5324254259156684
Stiri_pozitive/45.txt  0.5517954963923721     0.44820449471397406
Stiri_pozitive/11.txt  0.5920229834668777     0.40797702917719586
Stiri_pozitive/31.txt  0.28399709573761117     0.7160028996069903
Stiri_pozitive/3.txt  0.29301863369996073     0.7069813603662707
Stiri_pozitive/39.txt  0.48777835525135105     0.5122216430135199
Stiri_pozitive/17.txt  0.36437872724655346     0.6356212822290567
Stiri_pozitive/48.txt  0.3196603127375797     0.6803396973038407
Stiri_pozitive/8.txt  0.36890475714453763     0.6310952443429245
Stiri_pozitive/10.txt  0.5356142508565699     0.46438574493120466
Stiri_pozitive/16.txt  0.578829949721694     0.42117004652973267
Stiri_pozitive/47.txt  0.4030655147249464     0.59693448575635